# Machine learning for equipment failure
https://medium.com/swlh/machine-learning-for-equipment-failure-prediction-and-predictive-maintenance-pm-e72b1ce42da1

In [1]:
# import sys
# !{sys.executable} -m pip install xgboost imblearn plotly chart-studio

In [2]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly as plotly
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn import metrics

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import types
import pandas as pd

# def __iter__(self): return 0

## Import data

### Download data

In [3]:
# Remove the data if you run this notebook more than once
# !rm equipment_failure_data_1.csv

# import first half fom github
# !wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_1.csv

# Remove the data if you run this notebook more than once
# !rm equipment_failure_data_2.csv

# Import the second half from github
# !wget https://raw.githubusercontent.com/shadgriffin/machine_failure/master/equipment_failure_data_2.csv

### Load data

In [4]:
# Convert csv to pandas
pd_data_1 = pd.read_csv("equipment_failure_data_1.csv", sep=",", header=0)

# convert to pandas dataframe
pd_data_2 = pd.read_csv("equipment_failure_data_2.csv", sep=",", header=0)

# Concatenate the two data files into one dataframe
pd_data = pd.concat([pd_data_1, pd_data_2])

## Data exploration

In [5]:
pd_data.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
0,100001,12/2/14,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880
1,100001,12/3/14,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881
2,100001,12/4/14,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882
3,100001,12/5/14,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883
4,100001,12/6/14,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884


* ID - ID field that represents a specific machine.
* DATE - The date of the observation.
* REGION_CLUSTER - A field that represents the region in which the machine resides.
* MAINTENANCE_VENDOR - A field that represents the company that provides maintenance and service to the machine.
* MANUFACTURER - The company that manufactured the equipment in question.
* WELL_GROUP - A field representing the type of machine.
* EQUIPMENT_AGE - Age of the machine, in days.
* Sxx - Sensor values.
* EQUIPMENT_FAILURE - A '1' means that the equipment failed. A '0' means the equipment did not fail.

In [8]:
# Shape of the data
pd_data.shape

(307751, 16)

Num of machines: 421

In [7]:
num_machines = pd.DataFrame(pd_data.groupby(['ID']).agg(['count']))
num_machines.shape

(421, 15)

Num of unique datasets: 721

In [8]:
unique_datasets = pd.DataFrame(pd_data.groupby(['DATE']).agg(['count']))
unique_datasets.shape

(731, 15)

### Drop duplicates

In [9]:
df_failure = pd_data.copy()
df_failure = df_failure.drop_duplicates(subset=['ID', 'DATE'])
df_failure.shape

(307751, 16)

In [10]:
# Check for null values
pd_data.isnull().sum(axis=0)

ID                    0
DATE                  0
REGION_CLUSTER        0
MAINTENANCE_VENDOR    0
MANUFACTURER          0
WELL_GROUP            0
S15                   0
S17                   0
S13                   0
S5                    0
S16                   0
S19                   0
S18                   0
EQUIPMENT_FAILURE     0
S8                    0
AGE_OF_EQUIPMENT      0
dtype: int64

In [12]:
labels = pd.DataFrame(pd_data.groupby(['EQUIPMENT_FAILURE'])
                            ['ID'].agg('count'))
labels

,ID
EQUIPMENT_FAILURE,
0,307330
1,421


In [13]:
# Describe dataset
pd_data.describe()

,ID,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT
count,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000,307751.000000
mean,100310.826603,4.543943,14.585192,80.265541,35.018249,4675.848252,7.972097,9.069123,137.963064,0.001368,144.665715,2524.192399
std,177.574390,2.284121,8.817056,85.804273,14.446585,2521.074632,2.321949,16.898887,238.890128,0.036961,240.773926,3158.930976
min,100001.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-16.490000,0.000000
25%,100161.000000,3.000000,7.694100,0.000000,28.200000,3209.000000,6.621500,0.900000,11.798276,0.000000,9.250000,721.000000
50%,100311.000000,5.000000,11.661600,31.680000,34.940000,4237.047619,8.004000,4.200000,38.200000,0.000000,53.080000,1113.000000
75%,100467.000000,6.000000,22.560000,160.080000,41.610000,5743.000000,9.460000,10.600000,150.900000,0.000000,165.092608,2784.000000
max,100617.000000,8.000000,59.040000,2555.520000,592.890000,52767.000000,24.600000,511.000000,4151.700000,1.000000,2068.110000,15170.000000


## Data transformations and Feature engineering

### Create running summaries

In [14]:
# Convert dates from character to date
pd_data['DATE'] = pd.to_datetime(pd_data['DATE'])

In [15]:
# Flipper: Indicates when the ID changes as the data are sorted by ID and DATE in ascending order.
pd_data = pd_data.sort_values(by=['ID', 'DATE'], ascending=[True, True])

pd_data['flipper'] = np.where((pd_data.ID != pd_data.ID.shift(1)), 1, 0)
pd_data.head()

,ID,DATE,REGION_CLUSTER,MAINTENANCE_VENDOR,MANUFACTURER,WELL_GROUP,S15,S17,S13,S5,S16,S19,S18,EQUIPMENT_FAILURE,S8,AGE_OF_EQUIPMENT,flipper
0,100001,2014-12-02,G,O,Y,1,11.088000,145.223448,39.34,3501.0,8.426869,1.9,24.610345,0,0.0,880,1
1,100001,2014-12-03,G,O,Y,1,8.877943,187.573214,39.20,3489.0,6.483714,1.9,24.671429,0,0.0,881,0
2,100001,2014-12-04,G,O,Y,1,8.676444,148.363704,38.87,3459.0,6.159659,2.0,24.733333,0,0.0,882,0
3,100001,2014-12-05,G,O,Y,1,9.988338,133.660000,39.47,3513.0,9.320308,2.0,24.773077,0,0.0,883,0
4,100001,2014-12-06,G,O,Y,1,8.475264,197.181600,40.33,3589.0,8.022960,1.5,24.808000,0,0.0,884,0


In [19]:
# define your feature window. This is the window by which we will aggregate our sensor values
feature_window = 21

In [20]:
dfx = pd_data.copy()

# Select the first record of each machine
starter = dfx[dfx['flipper'] == 1]
starter = starter[['DATE', 'ID']]

# Rename date to start_date
starter = starter.rename(index=str, columns={"DATE": "START_DATE"})

# Convert START_DATE to date
starter['START_DATE'] = pd.to_datetime(starter['START_DATE'])

# Merge START_DATE to the original data set
dfx = dfx.sort_values(by=['ID', 'DATE'], ascending=[True, True])
starter = starter.sort_values(by=['ID'], ascending=[True])
dfx = dfx.merge(starter, on=['ID'], how='left')

# Calculate the number of days since the beginning of each well.
dfx['C'] = dfx['DATE'] - dfx['START_DATE']
dfx['TIME_SINCE_START'] = dfx['C'] / np.timedelta64(1, 'D')
dfx = dfx.drop(columns=['C'])
dfx['too_soon'] = np.where((dfx.TIME_SINCE_START < feature_window), 1, 0)

In [ ]:
# Create a running mean, max, min and median for the sensor variables.
dfx['S5_mean'] = np.where((dfx.too_soon == 0),
                          (dfx['S5'].rolling(min_periods=1, window=feature_window).mean(),
                           dfx.S5)
                          dfx